In [61]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [62]:
#Get the html source
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
#df.to_csv('beautifulsoup_pandas.csv',header=0,index=False)
print (df.head())

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront


In [63]:
#Remove rows with "Not assigned" values in Borough
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [64]:
#Assign the "Not assigned" value of Neighbourhood with the value of Borough
df.loc[df['Neighbourhood']=='Not assigned' , 'Neighbourhood'] =  df.loc[df['Neighbourhood']=='Not assigned' , 'Borough']

In [65]:
#df_g.shape
df.shape

(211, 3)

In [66]:
#Get the logngitute and latitude
!wget -q -O 'long_lat.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

long_lat = pd.read_csv('long_lat.csv')
#df.to_csv('beautifulsoup_pandas.csv',header=0,index=False)

# rename postal code column to Postcode

long_lat.rename(columns={'Postal Code':'Postcode'}, inplace=True)

print (long_lat.head())

Data downloaded!
  Postcode   Latitude  Longitude
0      M1B  43.806686 -79.194353
1      M1C  43.784535 -79.160497
2      M1E  43.763573 -79.188711
3      M1G  43.770992 -79.216917
4      M1H  43.773136 -79.239476


In [67]:
#Add the long and lat columns to the original dataframe
df_final = pd.merge(df, long_lat, how='inner', on = 'Postcode')
df_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [68]:
# Filter for Toronto
df_Toronto = df_final[df_final['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [69]:
address = 'Toronto'#'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [70]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [71]:
#Foursquare
CLIENT_ID = 'W0WJZTQLMIUHI1UTZLVVAYLK0RDXU4P3YR5A3OVXVZ41MKS1' # your Foursquare ID
CLIENT_SECRET = '2AZUPZIMYAEQ5LRLRINDLASGU1CH303F5U4443VAQYPMLJVV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W0WJZTQLMIUHI1UTZLVVAYLK0RDXU4P3YR5A3OVXVZ41MKS1
CLIENT_SECRET:2AZUPZIMYAEQ5LRLRINDLASGU1CH303F5U4443VAQYPMLJVV


In [72]:
## Get the University and Parks Neighbours only
Uni_park_Toronto = df_Toronto[df_Toronto['Neighbourhood'].str.contains("University")]
#Uni_airp_Toronto = pd.concat([Uni_airp_Toronto, df_Toronto[df_Toronto['Neighbourhood'].str.contains("airport")]] , ignore_index=True, axis=0)#, ignore_index=False)#, verify_integrity=False, sort=None)
Uni_park_Toronto = pd.concat([Uni_park_Toronto, df_Toronto[df_Toronto['Neighbourhood'].str.contains("Park")]] , ignore_index=True, axis=0)

Uni_park_Toronto


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5S,Downtown Toronto,University of Toronto,43.662696,-79.400049
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
3,M6K,West Toronto,Parkdale Village,43.636847,-79.428191
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M6P,West Toronto,High Park,43.661608,-79.464763
6,M6R,West Toronto,Parkdale,43.648960,-79.456325
7,M4T,Central Toronto,Moore Park,43.689574,-79.383160
8,M5T,Downtown Toronto,Grange Park,43.653206,-79.400049
9,M4V,Central Toronto,Deer Park,43.686412,-79.400049


In [15]:
#Let's create a function to repeat the same process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Toronto_venues = getNearbyVenues(names=Uni_park_Toronto['Neighbourhood'],
                                   latitudes=Uni_park_Toronto['Latitude'],
                                   longitudes=Uni_park_Toronto['Longitude']
                                  )

#Let's check the size of the resulting dataframe
print(Toronto_venues.shape)


University of Toronto
Regent Park
Berczy Park
Parkdale Village
Lawrence Park
High Park
Parkdale
Moore Park
Grange Park
Deer Park
(320, 7)


In [16]:
Toronto_venues[Toronto_venues['Neighborhood']=='Lawrence Park']

,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
160,Lawrence Park,43.72802,-79.38879,Lawrence Park Ravine,43.726963,-79.394382,Park
161,Lawrence Park,43.72802,-79.38879,Lake,43.727910,-79.386857,Lake
162,Lawrence Park,43.72802,-79.38879,Zodiac Swim School,43.728532,-79.382860,Swim School
163,Lawrence Park,43.72802,-79.38879,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line


In [17]:
#Let's check how many venues were returned for each neighborhood

Toronto_venues.groupby('Neighborhood').count()

#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques venues.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 128 uniques venues.


In [18]:
#Get the Restaurant Venues only
#restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
Rest_Toronto = pd.DataFrame(Toronto_venues[Toronto_venues['Venue Category'].str.contains("Restaurant")])
Rest_Toronto.head()#groupby('Neighborhood')

,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,University of Toronto,43.662696,-79.400049,Yasu,43.662837,-79.403217,Japanese Restaurant
1,University of Toronto,43.662696,-79.400049,Rasa,43.662757,-79.403988,Restaurant
2,University of Toronto,43.662696,-79.400049,Piano Piano,43.662949,-79.402898,Italian Restaurant
5,University of Toronto,43.662696,-79.400049,Cafe Cancan,43.662735,-79.403447,French Restaurant
11,University of Toronto,43.662696,-79.400049,Akai Sushi,43.662470,-79.404946,Sushi Restaurant


In [73]:
#Get Number of each restaurant vategory in each Neighborhood
Count_Rest = Rest_Toronto[['Neighborhood','Venue Category','Venue']].reset_index(drop=True).groupby(['Neighborhood','Venue Category']).count()
Count_Rest.rename(columns={'Venue':'Count'}, inplace=True)
Count_Rest

Count
Neighborhood          Venue Category                      
Berczy Park           Comfort Food Restaurant            1
                      French Restaurant                  1
                      Greek Restaurant                   1
                      Italian Restaurant                 1
                      Restaurant                         1
                      Seafood Restaurant                 2
                      Thai Restaurant                    1
                      Vegetarian / Vegan Restaurant      1
Deer Park             American Restaurant                1
                      Restaurant                         1
                      Sushi Restaurant                   1
                      Vietnamese Restaurant              1
Grange Park           Caribbean Restaurant               2
                      Chinese Restaurant                 5
                      Comfort Food Restaurant            2
                      Doner Restaurant                   1
                      Dumpling Restaurant                4
                      Filipino Restaurant                1
                      Italian Restaurant                 1
                      Japanese Restaurant                1
                      Mexican Restaurant                 4
                      Ramen Restaurant                   1
                      Thai Restaurant                    1
                      Vegetarian / Vegan Restaurant      6
                      Vietnamese Restaurant              4
High Park             Cajun / Creole Restaurant          1
                      Fast Food Restaurant               1
                      Italian Restaurant                 1
                      Mexican Restaurant                 2
                      Thai Restaurant                    1
Moore Park            Restaurant                         1
Parkdale              Cuban Restaurant                   1
                      Eastern European Restaurant        1
                      Italian Restaurant                 1
                      Restaurant                         1
Parkdale Village      Caribbean Restaurant               1
                      Falafel Restaurant                 1
                      Italian Restaurant                 1
                      Restaurant                         1
Regent Park           French Restaurant                  1
                      Italian Restaurant                 1
                      Mexican Restaurant                 2
                      Restaurant                         2
University of Toronto Chinese Restaurant                 1
                      Comfort Food Restaurant            1
                      French Restaurant                  1
                      Italian Restaurant                 1
                      Japanese Restaurant                2
                      Restaurant                         2
                      Sushi Restaurant                   1

In [49]:
#Get Number of each restaurant vategory in each Neighborhood
Count_All_Rest = Rest_Toronto[['Neighborhood','Venue']].reset_index(drop=True).groupby(['Neighborhood']).count()
Count_All_Rest.rename(columns={'Venue':'Count'}, inplace=True)
Count_All_Rest.reset_index(drop=False, inplace=True)
Count_All_Rest

,Neighborhood,Count
0,Berczy Park,9
1,Deer Park,4
2,Grange Park,33
3,High Park,6
4,Moore Park,1
5,Parkdale,4
6,Parkdale Village,4
7,Regent Park,6
8,University of Toronto,9


In [60]:
# download countries geojson file
#!wget --quiet https://github.com/jasonicarter/toronto-geojson/blob/master/toronto_topo.json -O toronto_topo.json
 
#!wget --quiet https://github.com/jasonicarter/toronto-geojson/blob/master/toronto_crs84.geojson -O toronto_crs84.geojson
    
#!wget --quiet https://github.com/codeforamerica/click_that_hood/blob/master/public/data/toronto.geojson -O toronto.geojson
#print('GeoJSON file downloaded!')

In [59]:
# create map of Toronto using latitude and longitude values
#map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
#toronto_geo = r'toronto.geojson' # geojson file

# generate choropleth map to show the Number of Restaurants in each Neighborhood
#map_toronto.choropleth(
#    geo_data=toronto_geo,
#    data=Count_All_Rest,
#    columns=['Neighborhood', 'Count'],
#    key_on='feature.properties.name',
#    fill_color='YlOrRd', 
#    fill_opacity=0.7, 
#    line_opacity=0.2,
#    legend_name='Number of Restaurants in each Neighborhood'
#)


In [20]:
#Analyze Each Neighborhood
# one hot encoding
Toronto_onehot = pd.get_dummies(Rest_Toronto[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Rest_Toronto['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,American Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,...,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,University of Toronto,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,University of Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,University of Toronto,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,University of Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,University of Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [21]:
Toronto_onehot.shape

(76, 25)

In [22]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped


,Neighborhood,American Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,...,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Berczy Park,0.00,0.000000,0.000000,0.000000,0.111111,0.00,0.000000,0.000000,0.00,...,0.111111,0.000000,0.000000,0.000000,0.111111,0.222222,0.000000,0.111111,0.111111,0.000000
1,Deer Park,0.25,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.250000,0.000000,0.000000,0.250000
2,Grange Park,0.00,0.000000,0.060606,0.151515,0.060606,0.00,0.030303,0.121212,0.00,...,0.030303,0.030303,0.121212,0.030303,0.000000,0.000000,0.000000,0.030303,0.181818,0.121212
3,High Park,0.00,0.166667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.166667,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000
4,Moore Park,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Parkdale,0.00,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.25,...,0.250000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Parkdale Village,0.00,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.250000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Regent Park,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.166667,0.000000,0.333333,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000
8,University of Toronto,0.00,0.000000,0.000000,0.111111,0.111111,0.00,0.000000,0.000000,0.00,...,0.111111,0.222222,0.000000,0.000000,0.222222,0.000000,0.111111,0.000000,0.000000,0.000000


In [23]:
#Let's confirm the new size
Toronto_grouped.shape

(9, 25)

In [24]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                           venue  freq
0             Seafood Restaurant  0.22
1              French Restaurant  0.11
2               Greek Restaurant  0.11
3  Vegetarian / Vegan Restaurant  0.11
4                Thai Restaurant  0.11


----Deer Park----
                     venue  freq
0      American Restaurant  0.25
1               Restaurant  0.25
2         Sushi Restaurant  0.25
3    Vietnamese Restaurant  0.25
4  Comfort Food Restaurant  0.00


----Grange Park----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.18
1             Chinese Restaurant  0.15
2          Vietnamese Restaurant  0.12
3             Mexican Restaurant  0.12
4            Dumpling Restaurant  0.12


----High Park----
                       venue  freq
0         Mexican Restaurant  0.33
1            Thai Restaurant  0.17
2       Fast Food Restaurant  0.17
3         Italian Restaurant  0.17
4  Cajun / Creole Restaurant  0.17


----Moore Park----
                     

In [25]:
#Let's put that into a pandas dataframe
#First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Thai Restaurant,Restaurant,Comfort Food Restaurant,Italian Restaurant,Greek Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Dumpling Restaurant
1,Deer Park,Vietnamese Restaurant,Sushi Restaurant,Restaurant,American Restaurant,Seafood Restaurant,Thai Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant
2,Grange Park,Vegetarian / Vegan Restaurant,Chinese Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Mexican Restaurant,Caribbean Restaurant,Comfort Food Restaurant,Italian Restaurant,Doner Restaurant,Filipino Restaurant
3,High Park,Mexican Restaurant,Thai Restaurant,Cajun / Creole Restaurant,Italian Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Falafel Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant
4,Moore Park,Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Dumpling Restaurant
5,Parkdale,Restaurant,Cuban Restaurant,Italian Restaurant,Eastern European Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant
6,Parkdale Village,Caribbean Restaurant,Restaurant,Italian Restaurant,Falafel Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Cajun / Creole Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant
7,Regent Park,Restaurant,Mexican Restaurant,Italian Restaurant,French Restaurant,Vietnamese Restaurant,Falafel Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant
8,University of Toronto,Restaurant,Japanese Restaurant,Sushi Restaurant,Chinese Restaurant,Comfort Food Restaurant,Italian Restaurant,French Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Cajun / Creole Restaurant


In [26]:
#Cluster Neighborhoods
#Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1).drop_duplicates()

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 



array([0, 3, 0, 2, 1, 4, 4, 2, 0], dtype=int32)

In [27]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_.astype(int))

Toronto_merged = Uni_park_Toronto

Toronto_merged.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#Toronto_merged = Toronto_merged.replace(np.nan, 'No Restaurants', regex=True)

Toronto_merged.dropna(inplace = True)

Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].astype(int)

Toronto_merged#.head() # check the last columns!



,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5S,Downtown Toronto,University of Toronto,43.662696,-79.400049,0,Restaurant,Japanese Restaurant,Sushi Restaurant,Chinese Restaurant,Comfort Food Restaurant,Italian Restaurant,French Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Cajun / Creole Restaurant
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,2,Restaurant,Mexican Restaurant,Italian Restaurant,French Restaurant,Vietnamese Restaurant,Falafel Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant
2,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Seafood Restaurant,Thai Restaurant,Restaurant,Comfort Food Restaurant,Italian Restaurant,Greek Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Dumpling Restaurant
3,M6K,West Toronto,Parkdale Village,43.636847,-79.428191,4,Caribbean Restaurant,Restaurant,Italian Restaurant,Falafel Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Cajun / Creole Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant
5,M6P,West Toronto,High Park,43.661608,-79.464763,2,Mexican Restaurant,Thai Restaurant,Cajun / Creole Restaurant,Italian Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Falafel Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant
6,M6R,West Toronto,Parkdale,43.648960,-79.456325,4,Restaurant,Cuban Restaurant,Italian Restaurant,Eastern European Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant
7,M4T,Central Toronto,Moore Park,43.689574,-79.383160,1,Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Dumpling Restaurant
8,M5T,Downtown Toronto,Grange Park,43.653206,-79.400049,0,Vegetarian / Vegan Restaurant,Chinese Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Mexican Restaurant,Caribbean Restaurant,Comfort Food Restaurant,Italian Restaurant,Doner Restaurant,Filipino Restaurant
9,M4V,Central Toronto,Deer Park,43.686412,-79.400049,3,Vietnamese Restaurant,Sushi Restaurant,Restaurant,American Restaurant,Seafood Restaurant,Thai Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant


In [28]:
#Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [78]:
#Examine Clusters
#Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#Cluster 1
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0]#, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5S,Downtown Toronto,University of Toronto,43.662696,-79.400049,0,Restaurant,Japanese Restaurant,Sushi Restaurant,Chinese Restaurant,Comfort Food Restaurant,Italian Restaurant,French Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Cajun / Creole Restaurant
2,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Seafood Restaurant,Thai Restaurant,Restaurant,Comfort Food Restaurant,Italian Restaurant,Greek Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Dumpling Restaurant
8,M5T,Downtown Toronto,Grange Park,43.653206,-79.400049,0,Vegetarian / Vegan Restaurant,Chinese Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Mexican Restaurant,Caribbean Restaurant,Comfort Food Restaurant,Italian Restaurant,Doner Restaurant,Filipino Restaurant


In [75]:
#Cluster 2
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1]#, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M4T,Central Toronto,Moore Park,43.689574,-79.38316,1,Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Dumpling Restaurant


In [76]:
#Cluster 3
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2]#, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,2,Restaurant,Mexican Restaurant,Italian Restaurant,French Restaurant,Vietnamese Restaurant,Falafel Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant
5,M6P,West Toronto,High Park,43.661608,-79.464763,2,Mexican Restaurant,Thai Restaurant,Cajun / Creole Restaurant,Italian Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Falafel Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant


In [77]:
#Cluster 4
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3]#, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M4V,Central Toronto,Deer Park,43.686412,-79.400049,3,Vietnamese Restaurant,Sushi Restaurant,Restaurant,American Restaurant,Seafood Restaurant,Thai Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant


In [74]:
#Cluster 5
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4]#, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M6K,West Toronto,Parkdale Village,43.636847,-79.428191,4,Caribbean Restaurant,Restaurant,Italian Restaurant,Falafel Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Cajun / Creole Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant
6,M6R,West Toronto,Parkdale,43.648960,-79.456325,4,Restaurant,Cuban Restaurant,Italian Restaurant,Eastern European Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant
